In [1]:
import pandas as pd
import neptune

NEPTUNE_API_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxMGM5ZDhiMy1kOTlhLTRlMTAtOGFlYy1hOTQzMDE1YjZlNjcifQ=='

PROJECT_ID = 'revivemed/Survival-RCC'

In [2]:
df_file = '/Users/jonaheaton/Desktop/Survival-RCC (4).csv'
df = pd.read_csv(df_file)
# chosen_columns = ['desc_str','sweep_kwargs/batch_size','sweep_kwargs/encoder_weight']

In [3]:
start_columns = df.columns
start_columns = start_columns.drop(['Creation Time','Tags','Owner','Id'])
val_columns = []
test_columns = []
param_columns = []
for x in start_columns:
    if 'avg_training_run' in x:
        val_columns.append(x)
        new_x = x.replace('avg_training_run/','avg_testing_run/')
        if 'val__' in new_x:
            new_x = new_x.replace('val__','test__')
            test_columns.append(new_x)
        else:
            test_columns.append(new_x)
    else:
        param_columns.append(x)

chosen_columns = param_columns + val_columns + test_columns

In [9]:
project = neptune.init_project(
    project=PROJECT_ID, 
    mode="read-only",
)

runs_table_df = project.fetch_runs_table(
    query='(`avg_training_run/num_success`:int >= 5)',
    columns = chosen_columns,
).to_pandas()

if 'sys/id' not in param_columns:
    param_columns = ['sys/id']	+ param_columns

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/


In [40]:
df = runs_table_df[param_columns + val_columns + test_columns].copy()

In [41]:
# create a two level column index
# first level is the type of data (param, val, test)
# second level is the column name

df[val_columns] = df[val_columns].round(3)
df[test_columns] = df[test_columns].round(3)

# create a list of tuples for the columns
skip_std = True

columns = []
skip_cols = []
for x in param_columns:
    x0 = x.replace('sweep_kwargs/','')
    x0 = x0.replace('_',' ')
    columns.append(('param',x0))

for x in val_columns:
    x0 = x.replace('avg_training_run/','')
    x0 = x0.replace('metrics/', '')
    x0 = x0.replace('val__','')
    x0 = x0.replace('_Concordance Index', ' C-Index')
    x0 = x0.replace('_AUROC (ovo, macro)', ' AUC')
    x0 = x0.replace('__', ' ')
    x0 = x0.replace('head_','head[')
    x0 = x0.replace(' on_', '] on ')
    # x0 = x0.replace('on_', ' on ')
    x0 = x0.replace('num_success','num iterations')

    if skip_std and 'std' in x0[:4]:
        skip_cols.append(x)
        continue
    columns.append(('val',x0))

for x in test_columns:
    x0 = x.replace('avg_testing_run/','')
    x0 = x0.replace('metrics/', '')
    x0 = x0.replace('test__','')
    x0 = x0.replace('_Concordance Index', ' C-Index')
    x0 = x0.replace('_AUROC (ovo, macro)', ' AUC')
    x0 = x0.replace('__', ' ')
    x0 = x0.replace('head_','head[')
    x0 = x0.replace(' on_', '] on ')
    # x0 = x0.replace('on_', ' on ')
    x0 = x0.replace('num_success','num iterations')
    if skip_std and 'std' in x0[:4]:
        skip_cols.append(x)
        continue
    columns.append(('test',x0))

# create a multiindex
columns = pd.MultiIndex.from_tuples(columns)

# set the columns
kept_cols = [x for x in df.columns if x not in skip_cols]
df = df[kept_cols].copy()
df.columns = columns

# round the values in the val and test columns to three decimal places
# df = df.round({('val',x):3 for x in val_columns})
# df = df.round({('test',x):3 for x in test_columns})

# clean up the 

In [42]:
# save to excel
df.to_excel('/Users/jonaheaton/Desktop/Survival-RCC-Filtered.xlsx')

## Assign names to architectures

In [53]:
layer1_0 = {
    'noise_factor': 0.25,
    'learning_rate': 0.0007869775056037999,
    'l2_reg_weight': 1.0092405183765013e-05,
    'l1_reg_weight': 3.137204254745065e-05,
    'num_epochs': 87,
    'head_hidden_layers': 1,
    'EVER-OS__weight': 10.0,
    'NIVO-OS__weight': 5.0,
    'name': 'layer 1.0'
}

list_layerS = [layer1_0]

############## Layer-R methods ############

layerR1_0 = {
    'noise_factor': 0.1,
    'learning_rate': 0.0006221023998363983,
    'l2_reg_weight': 0.0,
    'l1_reg_weight': 0.0025635844524779894,
    'num_epochs': 93,
    'encoder_weight': 1.0,
    'dropout_rate': 0.3,
    'adversarial_start_epoch': 0,
    'adversary_weight': 1.0,
    'head_hidden_layers': 1,
    'EVER-OS__weight': 10.0,
    'NIVO-OS__weight': 4.0,
    'name': 'layer-R 1.0'
}

list_layerR = [layerR1_0]


######## Basic methods ############

basic1_0 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.000541422,
    'noise_factor': 0.2,
    'num_epochs': 71,
    'batch_size': 64,
    'head_hidden_layers': 0,
    'name' : 'basic 1.0'
}

basic1_1 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.000541422,
    'noise_factor': 0.2,
    'num_epochs': 71,
    'batch_size': 32,
    'head_hidden_layers': 0,
    'name': 'basic 1.1'
}

basic1_2 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.000541422,
    'noise_factor': 0.2,
    'num_epochs': 71,
    'batch_size': 32,
    'head_hidden_layers': 0,
    'remove_nans': True,
    'name': 'basic 1.2'
}

basic2_0 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.0013,
    'noise_factor': 0.2,
    'num_epochs': 98,
    'batch_size': 32,
    'clean_batch': True,
    'head_hidden_layers': 0,
    'weight_decay': 0.000016,
    'name': 'basic 2.0'
}

basic2_1 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.0013,
    'noise_factor': 0.2,
    'num_epochs': 98,
    'batch_size': 32,
    'clean_batch': False,
    'head_hidden_layers': 0,
    'weight_decay': 0.000016,
    'remove_nans': True,
    'name': 'basic 2.1'
}

list_basicS = [basic1_0, basic1_1, basic1_2, basic2_0, basic2_1]

###### Basic-R methods ############

basicR1_0 = {
    'dropout_rate': 0.4,
    'encoder_weight': 0.75,
    'learning_rate': 0.0021,
    'noise_factor': 0.25,
    'num_epochs': 38,
    'batch_size': 32,
    'head_hidden_layers': 0,
    'weight_decay': 0.0,
    'remove_nans': True,
    'name': 'basic-R 1.0'
}

basicR1_1 = {
    'dropout_rate': 0.4,
    'encoder_weight': 0.75,
    'learning_rate': 0.0021,
    'noise_factor': 0.25,
    'num_epochs': 38,
    'batch_size': 32,
    'head_hidden_layers': 0,
    'weight_decay': 0.0,
    'remove_nans': False,
    'name': 'basic-R 1.1'
}

list_basicR = [basicR1_0, basicR1_1]



all_methods = list_layerS + list_layerR + list_basicS + list_basicR

In [56]:
for with_id in df[('param','sys/id')].to_list():

    run = neptune.init_run(project=PROJECT_ID,
                            api_token=NEPTUNE_API_TOKEN,
                            with_id=with_id)

    run_sweep = run['sweep_kwargs'].fetch()
    matches_name_list = []
    for method in all_methods:
        does_match = True

        for key,val in method.items():
            if key == 'name':
                continue
            if key in run_sweep:
                if run_sweep[key] != val:
                    does_match = False
        
        if does_match:
            print('we have a match to method', method['name'])
            run['sweep_kwargs/name'] = method['name']
            matches_name_list.append(method['name'])

    if len(matches_name_list) < 1:
        print(f'no match for {with_id}')

    run.stop()

    # break

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/e/SUR-2626
we have a match to method basic-R 1.0
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 1 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/Survival-RCC/e/SUR-2626/metadata
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/e/SUR-2625
we have a match to method basic-R 1.0
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 1 operations synced, thanks for waiting!
[neptun